In [38]:
from sklearn.preprocessing import (
    PolynomialFeatures,
    MinMaxScaler)
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    PoissonRegressor
    )
from sklearn.ensemble import (
    RandomForestRegressor
)
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import urllib.parse

In [92]:


base_url = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"
select_clause = "$select=created_date,closed_date,complaint_type,incident_zip,location_type,descriptor,status,resolution_description,borough"
where_clause = "$where=created_date>='2020-01-20'%20AND%20borough!=%27Unspecified%27"
limit_clause = "$limit=50000"

url = base_url + "?" + "&".join([select_clause, where_clause, limit_clause])
encoded_url = urllib.parse.quote(url, safe=':/?$&%=')

In [93]:
url

"https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$select=created_date,closed_date,complaint_type,incident_zip,location_type,descriptor,status,resolution_description,borough&$where=created_date>='2020-01-20'%20AND%20borough!=%27Unspecified%27&$limit=50000"

In [94]:
dat = pd.read_csv(encoded_url)

In [95]:
dat.shape

(50000, 9)

In [96]:
dat.head(20)

,created_date,closed_date,complaint_type,incident_zip,location_type,descriptor,status,resolution_description,borough
0,2024-02-08T12:00:00.000,NaN,Derelict Vehicles,11234.0,Street,Derelict Vehicles,Open,If the abandoned vehicle meets the criteria to...,BROOKLYN
1,2024-02-08T12:00:00.000,NaN,Derelict Vehicles,11228.0,Street,Derelict Vehicles,Open,If the abandoned vehicle meets the criteria to...,BROOKLYN
2,2024-02-08T12:00:00.000,NaN,Derelict Vehicles,10461.0,Street,Derelict Vehicles,Open,If the abandoned vehicle meets the criteria to...,BRONX
3,2024-02-08T02:03:09.000,NaN,Street Condition,11434.0,NaN,Pothole,Open,The Department of Transportation referred this...,QUEENS
4,2024-02-08T02:02:21.000,NaN,Street Condition,11429.0,NaN,Pothole,Open,The Department of Transportation referred this...,QUEENS
5,2024-02-08T02:00:55.000,2024-02-08T02:00:55.000,Street Condition,11427.0,NaN,Pothole,Closed,The Department of Transportation determined th...,QUEENS
6,2024-02-08T01:59:51.000,2024-02-08T01:59:51.000,Street Condition,11434.0,NaN,Pothole,Closed,The Department of Transportation determined th...,QUEENS
7,2024-02-08T01:59:03.000,NaN,Street Condition,11434.0,NaN,Pothole,Open,The Department of Transportation referred this...,QUEENS
8,2024-02-08T01:20:58.000,2024-02-08T01:48:39.000,Encampment,10456.0,Street/Sidewalk,NaN,Closed,The Police Department visited the location and...,BRONX
9,2024-02-08T01:20:17.000,NaN,Consumer Complaint,11220.0,Business,Supermarket or Grocery Store,In Progress,NaN,BROOKLYN


In [97]:
dat.groupby(['complaint_type'])['created_date'].count()

complaint_type
AHV Inspection Unit       4
APPLIANCE               330
Abandoned Bike           32
Abandoned Vehicle      1320
Air Quality             171
                       ... 
WATER LEAK              753
Water Conservation       50
Water Quality            25
Water System            881
Wood Pile Remaining       5
Name: created_date, Length: 143, dtype: int64

In [100]:
list(dat['resolution_description'])[-2]

'Service Request status for this request is available on the Department of Transportationâ\x80\x99s website. Please click the â\x80\x9cLearn Moreâ\x80\x9d link below.'